# Import Required Package

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.tokenize import sent_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords

## Load the Dataset

In [27]:
df = pd.read_csv("dataset.csv",encoding='latin-1')

In [28]:
from datasets import load_dataset

data_files = {"train":"dataset.csv"}


In [29]:
new_dataset = load_dataset("csv",data_files=data_files)

In [30]:
new_sample = new_dataset["train"].shuffle(seed=42).select(range(1000))
new_sample[:3]

{'Artical Id': [1920, 2370, 81],
 'Artical Title': ['Ireland set 155 run target for O',
  'Sri Lanka Kulasekara quits tests focus ODIs T20',
  "['PV Sindhu advances after Lakshya Sen"],
 'Artical Data': ['DHARAMSALA: Ireland set a competitive target of 155 for Oman in the fourth match of the World Twenty20, first round group A, here at the Himachal Pradesh Cricket Association Stadium on Wednesday night.After winning the toss and electing to bat first, Ireland made a good start and their three top-order batsmen took them to 112 in 15 overs but later they failed to maintain the tempo and were restricted to 154 for five.Openers captain William Porterfield and Paul Stirling scored 29 each while Gary Wilson top-scored with 38.Oman seamer Munis Ansari captured three wickets for 37.Ireland picked right-arm quick Craig Young in their three-pronged pace attack, and off-spinner Andy McBrine. They left out left-arm spinner George Dockrell and Andy Balbirnie.Â\xa0',
  'strong>NEW DELHI: Sri Lankan

In [37]:
for split in new_dataset.keys():
    assert len(new_dataset[split]) == len(new_dataset[split].unique("Artical Id"))

In [38]:
new_dataset

DatasetDict({
    train: Dataset({
        features: ['Artical Id', 'Artical Title', 'Artical Data'],
        num_rows: 3222
    })
})

In [39]:
new_dataset_clean = new_dataset["train"].train_test_split(train_size=0.8, seed=42)
# Rename the default "test" split to "validation"
new_dataset_clean["validation"] = new_dataset_clean.pop("test")
# Add the "test" set to our `DatasetDict`
# new_dataset_clean["test"] = new_dataset["test"]
new_dataset_clean

DatasetDict({
    train: Dataset({
        features: ['Artical Id', 'Artical Title', 'Artical Data'],
        num_rows: 2577
    })
    validation: Dataset({
        features: ['Artical Id', 'Artical Title', 'Artical Data'],
        num_rows: 645
    })
})

In [40]:
nd1["validation"] = new_dataset_clean["validation"]
nd2= new_dataset_clean["train"].train_test_split(train_size=0.7,seed=42)
nd2["validation"] = nd1["validation"]
nd2["test"] = nd2.pop("test")
nd2

DatasetDict({
    train: Dataset({
        features: ['Artical Id', 'Artical Title', 'Artical Data'],
        num_rows: 1803
    })
    validation: Dataset({
        features: ['Artical Id', 'Artical Title', 'Artical Data'],
        num_rows: 645
    })
    test: Dataset({
        features: ['Artical Id', 'Artical Title', 'Artical Data'],
        num_rows: 774
    })
})

## Preprocessing

In [3]:
df[:3]

,Artical Id,Artical Title,Artical Data
0,1,"[""'Your No. 28 jersey was ... ': PM Modi to ex...","[""Prime Minister Narendra Modi praised Rani Ra..."
1,2,"[""Salima Tete named India captain for women's ...",'Salima TeteNEW DELHI: The Indian women\'s ho...
2,3,['Sultan of Johor Cup: India claim bronze,"""(Hockey India Photo)Johor Bahru (Malaysia): ..."


In [36]:
df.head()

,Artical Id,Artical Title,Artical Data
0,1,"[""'Your No. 28 jersey was ... ': PM Modi to ex...","[""Prime Minister Narendra Modi praised Rani Ra..."
1,2,"[""Salima Tete named India captain for women's ...",'Salima TeteNEW DELHI: The Indian women\'s ho...
2,3,['Sultan of Johor Cup: India claim bronze,"""(Hockey India Photo)Johor Bahru (Malaysia): ..."
3,4,['Delhi SG Pipers rope in Argentine Lucas Tosc...,'Lucas ToscaniDelhi SG Pipers has signed Arge...
4,5,['Delhi SG Pipers acquire star Australian defe...,'Corey WeyerDelhi SG Pipers have signed Austr...


In [37]:
df.tail()

,Artical Id,Artical Title,Artical Data
3217,3218,Laptop ban hits Dubai for 11m weekend traveller,strong>DUBAI: Dubai International Airport and ...
3218,3219,Pak China relations not against any third coun...,"strong>BEIJING: Former Prime Minister, Shaukat..."
3219,3220,Uber grounds self driving cars after accid,strong>WASHINGTON: Uber has grounded its fleet...
3220,3221,New Development Bank plans joint investments i...,strong>BEIJING: The New Development Bank plans...
3221,3222,Google powered Startup Weekend energizing prou...,strong>KARACHI: Karachi-based technology incub...


### Removing Unwanted column

In [30]:
df.drop(['ï»¿Artical Id', 'Unnamed: 3'], inplace=True, axis=1)
df.tail()


,ï»¿Artical Id,Artical Title,Artical Data
3747,3218.0,Laptop ban hits Dubai for 11m weekend traveller,strong>DUBAI: Dubai International Airport and ...
3748,3219.0,Pak China relations not against any third coun...,"strong>BEIJING: Former Prime Minister, Shaukat..."
3749,3220.0,Uber grounds self driving cars after accid,strong>WASHINGTON: Uber has grounded its fleet...
3750,3221.0,New Development Bank plans joint investments i...,strong>BEIJING: The New Development Bank plans...
3751,3222.0,Google powered Startup Weekend energizing prou...,strong>KARACHI: Karachi-based technology incub...


### Handeling NA Values

In [38]:
df=df.dropna(axis=0)

In [39]:
df.head()

,Artical Id,Artical Title,Artical Data
0,1,"[""'Your No. 28 jersey was ... ': PM Modi to ex...","[""Prime Minister Narendra Modi praised Rani Ra..."
1,2,"[""Salima Tete named India captain for women's ...",'Salima TeteNEW DELHI: The Indian women\'s ho...
2,3,['Sultan of Johor Cup: India claim bronze,"""(Hockey India Photo)Johor Bahru (Malaysia): ..."
3,4,['Delhi SG Pipers rope in Argentine Lucas Tosc...,'Lucas ToscaniDelhi SG Pipers has signed Arge...
4,5,['Delhi SG Pipers acquire star Australian defe...,'Corey WeyerDelhi SG Pipers have signed Austr...


In [59]:
# df["Artical Data"].drop_duplicates()

0       ["Prime Minister Narendra Modi praised Rani Ra...
1        'Salima TeteNEW DELHI: The Indian women\'s ho...
2        "(Hockey India Photo)Johor Bahru (Malaysia): ...
3        'Lucas ToscaniDelhi SG Pipers has signed Arge...
4        'Corey WeyerDelhi SG Pipers have signed Austr...
                              ...                        
3199    strong>DUBAI: Dubai International Airport and ...
3200    strong>BEIJING: Former Prime Minister, Shaukat...
3201    strong>WASHINGTON: Uber has grounded its fleet...
3220    strong>BEIJING: The New Development Bank plans...
3221    strong>KARACHI: Karachi-based technology incub...
Name: Artical Data, Length: 3113, dtype: object

In [68]:
# df["Artical Data"].count()

3222

In [66]:
df['Artical Data'].isna().sum()

0

In [41]:
y = df['Artical Title']

### Sentence Tokenization 

In [42]:
# Sentence Tokenization 
sentences = []
for text in df['Artical Data']:
    sentences.extend(sent_tokenize(text))
sentences

['["Prime Minister Narendra Modi praised Rani Rampal former India women\'s hockey captain for her 16-year career and contributions to the sport.',
 "Rani who scored 205 goals in 254 matches retires as one of India's most decorated players and will now mentor sub-junior players.",
 'Modi highlighted her remarkable performance at the Tokyo 2021 Olympics.Read More"',
 " 'Salima TeteNEW DELHI: The Indian women\\'s hockey team has appointed Salima Tete as captain for the upcoming Asian Champions Trophy set to take place from November 11 to 20 at the Rajgir Hockey Stadium in Bihar.",
 'Navneet Kaur will serve as vice-captain.The tournament features India competing against China Japan Korea Malaysia and Thailand for regional supremacy.',
 'India will begin their campaign against Malaysia on November 11.Captain Tete expressed her enthusiasm stating "It\\\'s an incredible feeling to lead the team into another major tournament especially as defending champions.',
 'We have trained rigorously and

### Word Tokenization

In [43]:
from nltk.tokenize import wordpunct_tokenize
# wordpunct_tokenize(corpus)
words = []
for word in df['Artical Data']:
    words.extend(wordpunct_tokenize(word))
words

['["',
 'Prime',
 'Minister',
 'Narendra',
 'Modi',
 'praised',
 'Rani',
 'Rampal',
 'former',
 'India',
 'women',
 "'",
 's',
 'hockey',
 'captain',
 'for',
 'her',
 '16',
 '-',
 'year',
 'career',
 'and',
 'contributions',
 'to',
 'the',
 'sport',
 '.',
 'Rani',
 'who',
 'scored',
 '205',
 'goals',
 'in',
 '254',
 'matches',
 'retires',
 'as',
 'one',
 'of',
 'India',
 "'",
 's',
 'most',
 'decorated',
 'players',
 'and',
 'will',
 'now',
 'mentor',
 'sub',
 '-',
 'junior',
 'players',
 '.',
 'Modi',
 'highlighted',
 'her',
 'remarkable',
 'performance',
 'at',
 'the',
 'Tokyo',
 '2021',
 'Olympics',
 '.',
 'Read',
 'More',
 '"',
 "'",
 'Salima',
 'TeteNEW',
 'DELHI',
 ':',
 'The',
 'Indian',
 'women',
 "\\'",
 's',
 'hockey',
 'team',
 'has',
 'appointed',
 'Salima',
 'Tete',
 'as',
 'captain',
 'for',
 'the',
 'upcoming',
 'Asian',
 'Champions',
 'Trophy',
 'set',
 'to',
 'take',
 'place',
 'from',
 'November',
 '11',
 'to',
 '20',
 'at',
 'the',
 'Rajgir',
 'Hockey',
 'Stadium',
 

In [44]:
total_word = list(words)
len(total_word)

1210388

### Converting into Lower Case

In [45]:
from nltk import word_tokenize
lower_word = []
for word in df['Artical Data']:
    lower_word.extend(word_tokenize(word.lower()))
lower_word

['[',
 '``',
 'prime',
 'minister',
 'narendra',
 'modi',
 'praised',
 'rani',
 'rampal',
 'former',
 'india',
 'women',
 "'s",
 'hockey',
 'captain',
 'for',
 'her',
 '16-year',
 'career',
 'and',
 'contributions',
 'to',
 'the',
 'sport',
 '.',
 'rani',
 'who',
 'scored',
 '205',
 'goals',
 'in',
 '254',
 'matches',
 'retires',
 'as',
 'one',
 'of',
 'india',
 "'s",
 'most',
 'decorated',
 'players',
 'and',
 'will',
 'now',
 'mentor',
 'sub-junior',
 'players',
 '.',
 'modi',
 'highlighted',
 'her',
 'remarkable',
 'performance',
 'at',
 'the',
 'tokyo',
 '2021',
 'olympics.read',
 'more',
 "''",
 "'salima",
 'tetenew',
 'delhi',
 ':',
 'the',
 'indian',
 'women\\',
 "'s",
 'hockey',
 'team',
 'has',
 'appointed',
 'salima',
 'tete',
 'as',
 'captain',
 'for',
 'the',
 'upcoming',
 'asian',
 'champions',
 'trophy',
 'set',
 'to',
 'take',
 'place',
 'from',
 'november',
 '11',
 'to',
 '20',
 'at',
 'the',
 'rajgir',
 'hockey',
 'stadium',
 'in',
 'bihar',
 '.',
 'navneet',
 'kaur',


In [46]:
total_word = list(lower_word)
len(total_word)

1078464

### Removing Stop-Words

In [47]:
from nltk.corpus import stopwords
clean_word =[]
words_to_remove = [',', '?', '.', ':', '', '!', '@', '$', '%', '^', '*', '"', '(' , ')', '<', '>', '&', ';', '[', ']', '\\\n','``','\\n','\'\'','\'','\'s','\\']
# words_to_remove = [",", ".", ":", "\\/", "!", "@", "$", "%", "^", "*","(" , ")", "<", ">", "&", ";", "[", "]", "\n","'",""",]

words_to_remove.extend(stopwords.words('english'))
clean_word.extend([word.strip() for word in lower_word if word.strip() not in words_to_remove])
clean_word

['prime',
 'minister',
 'narendra',
 'modi',
 'praised',
 'rani',
 'rampal',
 'former',
 'india',
 'women',
 'hockey',
 'captain',
 '16-year',
 'career',
 'contributions',
 'sport',
 'rani',
 'scored',
 '205',
 'goals',
 '254',
 'matches',
 'retires',
 'one',
 'india',
 'decorated',
 'players',
 'mentor',
 'sub-junior',
 'players',
 'modi',
 'highlighted',
 'remarkable',
 'performance',
 'tokyo',
 '2021',
 'olympics.read',
 "'salima",
 'tetenew',
 'delhi',
 'indian',
 'women\\',
 'hockey',
 'team',
 'appointed',
 'salima',
 'tete',
 'captain',
 'upcoming',
 'asian',
 'champions',
 'trophy',
 'set',
 'take',
 'place',
 'november',
 '11',
 '20',
 'rajgir',
 'hockey',
 'stadium',
 'bihar',
 'navneet',
 'kaur',
 'serve',
 'vice-captain.the',
 'tournament',
 'features',
 'india',
 'competing',
 'china',
 'japan',
 'korea',
 'malaysia',
 'thailand',
 'regional',
 'supremacy',
 'india',
 'begin',
 'campaign',
 'malaysia',
 'november',
 '11.captain',
 'tete',
 'expressed',
 'enthusiasm',
 'sta

In [48]:
total_word = list(clean_word)
len(total_word)

610400